# Formation 2022  

##  Initiation à l’analyse de données Minion pour l'analyse de métagénome viraux

Created by J. Orjuela (DIADE-IRD), D. Filloux (PHIM-CIRAD) and A. Comte (PHIM-IRD) and from a formation by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-INRAE) given in Septembre 2021

Septembre 2022


### Our objectives in the following TP are : 
- explore the diversity of the metavirome of pineapple.
- reconstruct the complete genome sequencing of a novel member of the genus Vitivirus in the family Betaflexiviridae (subfamily Trivirinae) infecting pineapple.

# TP1 : BASECALLING and QC

## 1. Creating the folder, downloading data and so on

### 1.1 Data

Before starting, please download special data created for this practical training. Data are available on the from I-Trop server.

This data is the total RNAs extracted from pineapple leaf samples collected in Reunion Island. Nanopore sequencing was performed using a MinION portable device and the cDNA-PCR Barcoding kit.
This original dataset contained more than 4M reads. We choose to sample it for this formation. 

In [ ]:
mkdir -p ~/work/SG-ONT-2022/DATA
cd ~/work/SG-ONT-2022/DATA

# download sample data already basecalled
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/data.fastq

## 2. Basecalling 

When DNA molecules are sequenced, electrical signals are stocked on fast5 format files.

These signals need to be converted on standard fastq files to post-analysis.

Several training dataset models are usually used to convert fast5 to fastq. 


### 2.1 Basecalling with Guppy

Guppy is a data processing toolkit that contains the 'Oxford Nanopore Technologies' basecalling algorithms, and several bioinformatic post-processing features.

Basecalling with guppy can be launch using guppy tool. 

Guppy takes fast5 raw read files and transform electrical signal in fastq files.

We recommend to basecaller yours dataset using a GPU graphic card to obtain results quickly.

Documentation about how to run Guppy on I-trop GPU can be found on https://bioinfo.ird.fr/index.php/tutorials-fr/gpu/


In [ ]:
#To see all the documentation of guppy:
  guppy_basecaller --help

To run guppy you need to choose your configuration file accordingly to the flowcell and the kit you used for sequencing.

In [ ]:
#List supported flowcells and kits:
  guppy_basecaller --print_workflows

3 types of config files:
- sup : high accuracy, very slow
- hac : medium accuracy, medium recources needes
- fast : poor accuracy, very fast

In [ ]:
Guppy can also demultiplex and trim adaptaters or barcodes.

#### Basecalling command line <span style="color:red"> (Don't run it! ) </span>

In [ ]:
guppy_basecaller -c dna_r9.4.1_450bps_sup.cfg -i fast5/ -r -s output --num_callers 4 --gpu_runners_per_device 8 --min_qscore 6 --device cuda:2 --trim_adapters --detect_adapter --detect_mid_strand_adapter --pt_scaling --do_read_splitting

## 3. Quality Control on Long Reads


Calculating data quantity

In [1]:
cd ~/work/SG-ONT-2022/DATA
pwd

/home/jovyan/work/SG-ONT-2022/DATA


Calculating how many reads are in the fastq file

In [2]:
awk '{s++}END{print s/4}' data.fastq

906090


Calculating how many bases were sequenced

In [3]:
seqtk seq -A data.fastq | grep -v ">" | wc -m

595284620


What is the sequencing depth?

### 3.1 Quality Control of FASTQ with Nanoplot

Control reads quality using Nanoplot. You can parameter this tool using --help.

In [4]:
NanoPlot --help

usage: NanoPlot [-h] [-v] [-t THREADS] [--verbose] [--store] [--raw] [--huge]
                [-o OUTDIR] [--no_static] [-p PREFIX] [--tsv_stats]
                [--info_in_report] [--maxlength N] [--minlength N]
                [--drop_outliers] [--downsample N] [--loglength]
                [--percentqual] [--alength] [--minqual N] [--runtime_until N]
                [--readtype {1D,2D,1D2}] [--barcoded] [--no_supplementary]
                [-c COLOR] [-cm COLORMAP]
                [-f [{png,jpg,jpeg,webp,svg,pdf,eps,json} [{png,jpg,jpeg,webp,svg,pdf,eps,json} ...]]]
                [--plots [{kde,hex,dot} [{kde,hex,dot} ...]]]
                [--legacy [{kde,dot,hex} [{kde,dot,hex} ...]]] [--listcolors]
                [--listcolormaps] [--no-N50] [--N50] [--title TITLE]
                [--font_scale FONT_SCALE] [--dpi DPI] [--hide_stats]
                (--fastq file [file ...] | --fasta file [file ...] | --fastq_rich file [file ...] | --fastq_minimal file [file ...] | --summary fi

Launch NanoPlot. You can launch NanoPlot using summaries or fastq files.

In [5]:
# create a folder to save results
mkdir -p ~/work/SG-ONT-2022/QC
cd ~/work/SG-ONT-2022/QC

In [6]:
###### run nanoplot 
NanoPlot -t 1 --fastq  ../DATA/data.fastq --outdir NANOPLOT

Check stats on created NanoStats file.

In [7]:
cat NANOPLOT/NanoStats.txt

General summary:         
Mean read length:                  656.0
Mean read quality:                  11.2
Median read length:                463.0
Median read quality:                11.4
Number of reads:               906,090.0
Read length N50:                   823.0
STDEV read length:                 488.3
Total bases:               594,378,530.0
Number, percentage and megabases of reads above quality cutoffs
>Q5:	906018 (100.0%) 594.4Mb
>Q7:	835966 (92.3%) 557.4Mb
>Q10:	624742 (68.9%) 452.8Mb
>Q12:	363812 (40.2%) 310.6Mb
>Q15:	48881 (5.4%) 57.8Mb
Top 5 highest mean basecall quality scores and their read lengths
1:	90.0 (170)
2:	33.5 (168)
3:	30.5 (205)
4:	30.0 (439)
5:	28.1 (42)
Top 5 longest reads and their mean basecall quality score
1:	28048 (6.5)
2:	9381 (8.9)
3:	8696 (9.0)
4:	7423 (10.1)
5:	6749 (8.6)


* What do you think about data? 

* Estimate coverage.

* What about reads quality qscore?

Observe NanoPlot-report.html.

To open it on jupyter you need to click on "trust HTML".

* what about this dataset?

### 3.2 Compare reads QC statistics with NanoComp (Summaries dataset)

Compare long reads sequencing datasets using **NanoComp**.

NanoComp compiles quality information in a useful html report.

You can launch NanoComp using summaries or fastq files.

In [ ]:
NanoComp --help

In [ ]:
NanoComp --fastq ../DATA/data.fastq --outdir NANOCOMP

###  Others complemental tools: 

https://github.com/wdecoster/NanoPlot#companion-scripts

* NanoComp: comparing multiple runs

* NanoStat: statistic summary report of reads or alignments

* NanoFilt: filtering and trimming of reads

* NanoLyse: removing contaminant reads (e.g. lambda control DNA) from fastq

* FiltLong : filtering long reads by quality https://github.com/rrwick/Filtlong


## 4. BONUS: relaunch QC tools on the original dataset

As descripted in 1.1 The dataset used previously is a sample of the original dataset.

In [ ]:
# download sequencing summary of the original data

mkdir -p ~/SG-ONT-2022/DATA/real_summaries/
cd ~/SG-ONT-2022/DATA/real_summaries/
wget --no-check-certificat -rm -nH --cut-dirs=1 --reject="index.html*" https://itrop.ird.fr/ont-training-2022/sequencing_summary.txt

Relaunch NanoComp and NanoPlot on this sequencing summary

In [ ]:
NanoComp --summary sequencing_summary.txt --outdir NANOCOMP_real
NanoPlot --summary sequencing_summary.txt --outdir NANOPLOT_real

Compare the outputs with the sampled ones.